# Data selection and resource access

In this notebook you will:
- filter and collect resources based on properties in the metadata
- carry out simple analysis tasks on custom data collections such as counting files or words

In [ ]:
# Make sure that the libraries we will need in this notebook are installed
!pip install lxml numpy matplotlib

# Define some handy defaults
nsmap = {"cmd": "http://www.clarin.eu/cmd/1",
         "cmdp_text": "http://www.clarin.eu/cmd/1/profiles/clarin.eu:cr1:p_1633000337997"}

from os.path import expanduser
data_dir = expanduser("~") + '/data'

## Retrieving metadata files

In [ ]:
import requests
from io import BytesIO
from zipfile import ZipFile

'''
    This function gets and unpacks a metadata archive
'''
def unpack_metadata(set_id, target_dir):
    # Construct the address of the .zip file with the metadata for one set
    md_zip_url = f'https://europeana-oai.clarin.eu/metadata/fulltext-aggregation/{set_id}.zip'
    
    # Retrieve the .zip file
    print(f'Retrieving {md_zip_url}')
    resp = requests.get(md_zip_url)
    zipfile = ZipFile(BytesIO(resp.content))
    
    # Uncompress the .zip into the target directory
    print(f'Extracting content in {target_dir}')
    zipfile.extractall(path=target_dir)
    print('Done')
    
# We now call the function for the set of Polish newspapers and 
# extract them in a temporary directory so that we can use it.
import os
set_id = '9200357' # Poland
metadata_dir = f'{os.path.expanduser("~")}/work/temp/metadata/{set_id}'
unpack_metadata(set_id, metadata_dir)

## Data segmentation using metadata properties
Now we have the metadata available, we want to use the properties in the metadata to create useful data segments. The metadata tells us, for instance, the year of publication of each newspaper. 

Thus, we can narrow down our selection of text files and at the same time, using the resource identifiers that are also found in the metadata, create a 'map' that allows us to look up specific files based on a property we are interested in - in this case, the year of publication.

In [ ]:
from lxml import etree

'''
    This function returns the publication year and resource identifiers for a metadata record.
    If the record is not a text resource record, it returns nothing
'''
def get_year_and_ids(record_file):
    xml_tree = etree.parse(record_file)
    text_resource_root_node = xml_tree.xpath('//cmd:Components/cmdp_text:TextResource', namespaces=nsmap)

    if len(text_resource_root_node) < 1:
        # Metadata in file is not for a text resource...
        return None
    else:
        # Get the temporal coverage information
        year_text_node = xml_tree.xpath(
            '//cmdp_text:TextResource/cmdp_text:TemporalCoverage/cmdp_text:Start/cmdp_text:year/text()', namespaces=nsmap)
        if len(year_text_node) < 1:
            # No publication year found... this is unexpected!
            print(f'Warning: year information not found in {record_file}')
            return None
        else:
            year = year_text_node[0]
            # get the record identifiers
            record_ids_text_nodes = xml_tree.xpath(
                '//cmdp_text:SubresourceDescription/cmdp_text:IdentificationInfo/cmdp_text:identifier/text()', namespaces=nsmap)
            
            # we only want the numeric identifiers...
            record_numeric_ids = [id for id 
                                  in record_ids_text_nodes 
                                  if id.isnumeric()]
            
            # return the information found in the record as a tuple
            return(year, record_numeric_ids)    


# Trying it out on a single file that we know to be there...
sample_file = f'{metadata_dir}/Ilustrowany_Dziennik_Ludowy_1939.xml'
get_year_and_ids(sample_file)

In [ ]:
'''
    Now we apply the function to all metadata files and construct an 'index'
    The index will be a dictionary that has the year as 'key' and a list of 
'''
index = {}
years_of_interest = range(1900, 1920)
for filename in os.listdir(metadata_dir):
    if filename.endswith('.xml'):
        info = get_year_and_ids(f'{metadata_dir}/{filename}')
        # did we in fact get metadata for a text resource?
        if info:
            # break down the info
            (year, ids) = info
            if int(year) in years_of_interest: # int(year) gives us a numeric value needed to check against range
                # add to our index
                if not year in index: # does the index have an entry for the year?
                    # if not, make the entry
                    index[year] = []
                # append the identifiers to the entry
                index[year] += ids

# summarize our index: (year, id count)
[(year, len(index[year]))for year in sorted(index.keys())]

## Accessing the resources for simple data analysis
In this section we will go beyond the metadata and access the text resource files themselves. The identifiers found in the metadata can be looked up to reliably locate the resource file. Once we know where to find the file, we can read it programmatically, which is the basis for any kind of analysis or further processing.

In [ ]:
'''
    Here we will prepare the look-up of resource files based on resource identifier
'''
import json
with open(f'{data_dir}/{set_id}/id_file_map.json', 'r') as id_filename_map_file:
    id_filename_map = json.load(id_filename_map_file)

'''
    A function that looks up the full path to a text resource file based on an identifier
    within the current set. If there is no mapping to a file, it returns nothing.
'''
def get_resource_file(identifier):
    if identifier in id_filename_map:
        filename = id_filename_map[identifier]
        return f'{data_dir}/{set_id}/{filename}'


# Trying it out on the first identifier for a specific year in our index
get_resource_file(index['1900'][0])

### Defining a simple analysis

In [ ]:
'''
    A function that carries out a very simple analysis on a file: count the words.
    Here we define word as any white-space separated group of one or more characters.
'''
def count_words(filename):
    if os.path.exists(filename):
        # open the file
        with open(filename, 'r') as file:
            # gather the number of words per line
            words_per_line = [len(str.split(line)) # splitting the text of the line and counting the items does the job for us!
                              for line
                              in file.readlines()]
            # return the sum
            return sum(words_per_line)

# Try it out on a random file
test_filename = get_resource_file(index['1900'][0])
count_words(test_filename)

In [ ]:
'''
    Now let's do this on all files in our index, and store the results per year.
    
    Note: this can take a while!
'''

# We use the 'NumPy' library because this makes it much easier to work with numeric data arrays
import numpy as np

counts_index = {}
for year in sorted(index.keys()):
    counts = [count_words(get_resource_file(identifier)) 
                          for identifier
                          in index[year]]
    
    # Wrap the counts in a NumPy array
    np_counts = np.array(counts)

    # Finally, exclude 'None' and '0' counts (which happen if the resource is not available)
    # and store the result in the index
    counts_index[year] = np_counts[np_counts.nonzero()]

print('Done')

### Simple statistics on our data

In [ ]:
'''
    Now we can get some statistics out of our index counts!
'''
print('Total words for each year:')
print([(year, counts_index[year].sum()) for year in counts_index])

print('')

print('Maximum number of words per issue for each year:')
print([(year, counts_index[year].max()) for year in counts_index])

print('')

print('Average words per issue for each year:')
print([(year, round(counts_index[year].mean())) for year in counts_index])


### Data visualization example

In [ ]:
'''
    We're using Pyplot to plot a chart of the average issue word count per year in our data set
'''

import matplotlib.pyplot as plt

years = [int(year) for year in counts_index.keys()]
averages = [round(counts_index[year].mean()) for year in counts_index]

# create a figure
fig, ax = plt.subplots()

# position data on axes
ax.bar(years, averages)

# set labels and tweak label rendering on axes
plt.xlabel('Year')
plt.ylabel('Word count (avg)')
plt.xticks(years, counts_index.keys(), rotation=45)

# show the plot
plt.show()